In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
from dateutil.relativedelta import relativedelta

vamos carregar os dados

In [2]:
df_limites = pd.read_excel('gastos_limites.xlsx', sheet_name='limites')
df_gastos = pd.read_excel('gastos_limites.xlsx', sheet_name='gastos')

Limpar os dados do limite

In [3]:
df_limites['dt_fechamento'] = pd.to_datetime(df_limites.dt_fechamento, format="%d-%m-%y")
df_limites['limite'] = df_limites['limite'].str.replace('$','', regex=False)
df_limites['limite'] = df_limites['limite'].str.replace('R ','')
df_limites['limite'] = df_limites['limite'].str.replace(',','.')
df_limites['limite'] = pd.to_numeric(df_limites['limite'])
df_limites['cliente'] = df_limites['cliente'].str.title()
df_limites['cliente'] = df_limites['cliente'].str.strip()

In [4]:
df_limites.isnull().sum()

Unnamed: 0       0
cliente          0
limite           0
dt_fechamento    0
dtype: int64

Limpar os dados dos gastos

In [5]:
df_gastos['dt_operacao'] = pd.to_datetime(df_gastos.dt_operacao, format="%d-%m-%y")
df_gastos['valor_operacao'] = df_gastos['valor_operacao'].str.replace('$','', regex=False)
df_gastos['valor_operacao'] = df_gastos['valor_operacao'].str.replace('R ','')
df_gastos['valor_operacao'] = df_gastos['valor_operacao'].str.replace(',','.')
df_gastos['valor_operacao'] = pd.to_numeric(df_gastos['valor_operacao']) 
df_gastos['mes_atual'] = pd.to_datetime(df_gastos.mes_atual, format='%b/%y')
df_gastos['nm_cliente'] = df_gastos['nm_cliente'].str.title()
df_gastos['nm_cliente'] = df_gastos['nm_cliente'].str.strip()

In [6]:
inverteNome = lambda s : s.split(', ')[1] + ' ' +  s.split(', ')[0] if ', ' in s else s


In [7]:
df_gastos['nm_cliente'] = df_gastos['nm_cliente'].apply(inverteNome)
df_limites['cliente'] = df_limites['cliente'].apply(inverteNome)

In [8]:
df_gastos.isnull().sum()

Unnamed: 0        0
nm_cliente        0
valor_operacao    0
num_parcelas      0
dt_operacao       0
mes_atual         0
dtype: int64

In [9]:
df_limites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     20 non-null     int64         
 1   cliente        20 non-null     object        
 2   limite         20 non-null     float64       
 3   dt_fechamento  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 768.0+ bytes


In [10]:
df_limites.head()

,Unnamed: 0,cliente,limite,dt_fechamento
0,0,João Silva,2575.0,2021-01-25
1,1,José Silva,4500.0,2021-01-05
2,2,Antônio Oliveira,2325.0,2021-01-25
3,3,Francisco Santos,375.0,2021-01-25
4,4,Carlos Rodrigues,2675.0,2021-01-05


In [11]:
df_gastos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      150 non-null    int64         
 1   nm_cliente      150 non-null    object        
 2   valor_operacao  150 non-null    float64       
 3   num_parcelas    150 non-null    int64         
 4   dt_operacao     150 non-null    datetime64[ns]
 5   mes_atual       150 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 7.2+ KB


In [12]:
df_gastos['dt_operacao'].unique()

array(['2021-01-01T00:00:00.000000000', '2021-01-02T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-04T00:00:00.000000000',
       '2021-01-05T00:00:00.000000000', '2021-01-06T00:00:00.000000000',
       '2021-01-07T00:00:00.000000000', '2021-01-08T00:00:00.000000000',
       '2021-01-09T00:00:00.000000000', '2021-01-10T00:00:00.000000000',
       '2021-01-11T00:00:00.000000000', '2021-01-12T00:00:00.000000000',
       '2021-01-13T00:00:00.000000000', '2021-01-14T00:00:00.000000000',
       '2021-01-15T00:00:00.000000000', '2021-01-16T00:00:00.000000000',
       '2021-01-17T00:00:00.000000000', '2021-01-18T00:00:00.000000000',
       '2021-01-19T00:00:00.000000000', '2021-01-20T00:00:00.000000000',
       '2021-01-21T00:00:00.000000000', '2021-01-22T00:00:00.000000000',
       '2021-01-23T00:00:00.000000000', '2021-01-24T00:00:00.000000000',
       '2021-01-25T00:00:00.000000000', '2021-01-26T00:00:00.000000000',
       '2021-01-27T00:00:00.000000000', '2021-01-28

In [13]:
df_gastos_acum = df_gastos.groupby(df_gastos['dt_operacao'].dt.strftime('%B'))[['nm_cliente','valor_operacao']]

In [14]:
df_gastos_acum.head()

,nm_cliente,valor_operacao
0,Paulo Gomes,855.00
1,Camila Santos,504.18
2,Márcia Silva,970.00
3,Maria Alves,2010.31
4,Lucas Oliveira,465.00


In [15]:
df_gastos['valor_parcela']=round((df_gastos['valor_operacao']/df_gastos['num_parcelas']),2)

In [16]:
df_limites.sort_values(by='cliente').head(10)

,Unnamed: 0,cliente,limite,dt_fechamento
12,12,Adriana Pereira,1875.0,2021-01-05
17,17,Aline Rodrigues,2600.0,2021-01-15
11,11,Ana Costa,5375.0,2021-01-25
2,2,Antônio Oliveira,2325.0,2021-01-25
19,19,Camila Santos,3275.0,2021-01-15
4,4,Carlos Rodrigues,2675.0,2021-01-05
15,15,Fernanda Oliveira,2925.0,2021-01-05
3,3,Francisco Santos,375.0,2021-01-25
1,1,José Silva,4500.0,2021-01-05
0,0,João Silva,2575.0,2021-01-25


In [17]:
df_gastos.sort_values(by='nm_cliente').head(10)

,Unnamed: 0,nm_cliente,valor_operacao,num_parcelas,dt_operacao,mes_atual,valor_parcela
74,74,Adriana Pereira,63.88,1,2021-01-14,2021-01-01,63.88
145,145,Adriana Pereira,1084.58,10,2021-01-30,2021-01-01,108.46
118,118,Adriana Pereira,1464.84,2,2021-01-25,2021-01-01,732.42
71,71,Adriana Pereira,2036.00,3,2021-01-14,2021-01-01,678.67
135,135,Adriana Pereira,2056.00,1,2021-01-28,2021-01-01,2056.00
126,126,Adriana Pereira,247.98,3,2021-01-26,2021-01-01,82.66
68,68,Adriana Pereira,764.42,3,2021-01-13,2021-01-01,254.81
130,130,Adriana Pereira,155.56,4,2021-01-27,2021-01-01,38.89
33,33,Aline Rodrigues,234.00,5,2021-01-06,2021-01-01,46.80
56,56,Aline Rodrigues,281.00,1,2021-01-10,2021-01-01,281.00


In [18]:
df_total = df_limites.merge(df_gastos, left_on='cliente', right_on='nm_cliente', how='left')
df_total.columns


Index(['Unnamed: 0_x', 'cliente', 'limite', 'dt_fechamento', 'Unnamed: 0_y',
       'nm_cliente', 'valor_operacao', 'num_parcelas', 'dt_operacao',
       'mes_atual', 'valor_parcela'],
      dtype='object')

In [19]:
len(df_total['cliente'].unique().tolist())

20

In [20]:
len(df_gastos['nm_cliente'].unique().tolist())

20

In [21]:
df_total.drop(df_total.columns[df_total.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [22]:
df_total.head(3)

,cliente,limite,dt_fechamento,nm_cliente,valor_operacao,num_parcelas,dt_operacao,mes_atual,valor_parcela
0,João Silva,2575.0,2021-01-25,João Silva,243.42,5,2021-01-03,2021-01-01,48.68
1,João Silva,2575.0,2021-01-25,João Silva,1121.00,8,2021-01-15,2021-01-01,140.12
2,João Silva,2575.0,2021-01-25,João Silva,611.00,4,2021-01-16,2021-01-01,152.75


In [23]:
df_total.columns

Index(['cliente', 'limite', 'dt_fechamento', 'nm_cliente', 'valor_operacao',
       'num_parcelas', 'dt_operacao', 'mes_atual', 'valor_parcela'],
      dtype='object')

In [24]:
df_total_com_parcelas = pd.DataFrame(columns=['cliente', 'limite', 'dt_pagamento', 'valor_operacao',
       'num_parcelas', 'num_parcela','dt_operacao', 'valor_parcela'])

In [25]:
df_total.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   cliente         150 non-null    object        
 1   limite          150 non-null    float64       
 2   dt_fechamento   150 non-null    datetime64[ns]
 3   nm_cliente      150 non-null    object        
 4   valor_operacao  150 non-null    float64       
 5   num_parcelas    150 non-null    int64         
 6   dt_operacao     150 non-null    datetime64[ns]
 7   mes_atual       150 non-null    datetime64[ns]
 8   valor_parcela   150 non-null    float64       
dtypes: datetime64[ns](3), float64(3), int64(1), object(2)
memory usage: 11.7+ KB


In [26]:
df_total.isna().sum()

cliente           0
limite            0
dt_fechamento     0
nm_cliente        0
valor_operacao    0
num_parcelas      0
dt_operacao       0
mes_atual         0
valor_parcela     0
dtype: int64

In [27]:
for i in df_total.values:
    for n in range(int(i[5])):
        
        limite = i[1]
        vencim = i[2]
        cliente = i[0]
        valor= i[4]
        num_parcelas = i[5]
        parcela=i[8]
        data_operacao=i[6]
        mes = 0
        data_pagamento = dt.datetime(data_operacao.year, data_operacao.month,vencim.day)
        data_pagamento = data_pagamento + relativedelta(months=n)
        if data_operacao.day > vencim.day and n==0:
            data_pagamento = data_pagamento + relativedelta(months=1)
        
        df_total_com_parcelas.loc[0 if pd.isnull(df_total_com_parcelas.index.max()) else df_total_com_parcelas.index.max() + 1] = [cliente, limite, data_operacao, valor,
            num_parcelas, str(n+1), data_pagamento, parcela]
        #print(i)
    


In [28]:
df_total_com_parcelas

,cliente,limite,dt_pagamento,valor_operacao,num_parcelas,num_parcela,dt_operacao,valor_parcela
0,João Silva,2575.0,2021-01-03,243.42,5,1,2021-01-25,48.68
1,João Silva,2575.0,2021-01-03,243.42,5,2,2021-02-25,48.68
2,João Silva,2575.0,2021-01-03,243.42,5,3,2021-03-25,48.68
3,João Silva,2575.0,2021-01-03,243.42,5,4,2021-04-25,48.68
4,João Silva,2575.0,2021-01-03,243.42,5,5,2021-05-25,48.68
...,...,...,...,...,...,...,...,...
564,Camila Santos,3275.0,2021-01-29,203.01,3,2,2021-02-15,67.67
565,Camila Santos,3275.0,2021-01-29,203.01,3,3,2021-03-15,67.67
566,Camila Santos,3275.0,2021-01-30,702.00,3,1,2021-02-15,234.00
567,Camila Santos,3275.0,2021-01-30,702.00,3,2,2021-02-15,234.00


In [29]:
result = df_total_com_parcelas.groupby([df_total_com_parcelas['dt_pagamento'].dt.year, df_total_com_parcelas['dt_pagamento'].dt.month, df_total_com_parcelas['cliente']]).agg({'valor_parcela':sum})

In [30]:
result

valor_parcela
dt_pagamento dt_pagamento cliente                         
2021         1            Adriana Pereira          7873.30
                          Aline Rodrigues          1649.12
                          Ana Costa                4907.02
                          Antônio Oliveira         5028.80
                          Camila Santos            4111.14
                          Carlos Rodrigues         5367.84
                          Fernanda Oliveira        2566.49
                          Francisco Santos         5330.69
                          José Silva               3863.07
                          João Silva               4839.44
                          Juliana Costa            4120.91
                          Lucas Oliveira           5269.59
                          Luiz Alves               6127.03
                          Marcos Souza             5869.66
                          Maria Alves              5524.50
                          Márcia Silva             8289.81
                          Patrícia Pereira         2171.39
                          Paulo Gomes             16559.84
                          Pedro Ribeira            2095.40
                          Sandra Souza             6137.77